In [1]:
import numpy as np
from pyts.multivariate.transformation import WEASELMUSE
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.metrics import *
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from wildboar.tree import ShapeletTreeClassifier
from sktime.classification.dictionary_based import MUSE
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.hybrid import HIVECOTEV2

In [2]:
def evaluate_model(model, X, y):
    pred_prob = model.predict_proba(X)[:,1] #predicted classes
    fpr, tpr, _ = roc_curve(y, pred_prob) # roc_curve
    auc_value = auc(fpr,tpr) # auc_value
    auc_rf =average_precision_score(y, pred_prob)
    y_pred = model.predict(X)
    MCC=matthews_corrcoef(y, y_pred)
    accuracy = accuracy_score(y,y_pred) # calculate accuracy
    report = classification_report(y, y_pred, labels=[0,1], output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df = report_df.reset_index()
    model_eval  = report_df[report_df['index'].str.contains('1')][['precision','recall','f1-score']]
    model_eval['accuracy']  = accuracy
    # list(report_df[report_df['index'].str.contains('accuracy')]['support'])
    model_eval['ROC']  = auc_value
    model_eval['PR']  = auc_rf
    model_eval['MCC']  = MCC
    # model_eval['time'] = time
    cf_matrix = confusion_matrix(y, y_pred)
    
    return model_eval, cf_matrix

def model_eval_data(model, X_train, y_train, 
                         X_test, y_test, 
                         model_eval_train, 
                         model_eval_test,
                         Name=None):
    # start = datetime.datetime.now()
    temp_eval_train, cf_matrix_train = evaluate_model(model, X_train, y_train)
    temp_eval_test, cf_matrix_test = evaluate_model(model, X_test, y_test)
    temp_eval_train.index = [Name]
    temp_eval_test.index = [Name]
    # end = datetime.datetime.now()
    # time=(end -start)*1000
    try:
        model_eval_train=pd.concat([model_eval_train,temp_eval_train],axis=0)
        model_eval_test=pd.concat([model_eval_test,temp_eval_test],axis=0)
    except:
        model_eval_train = temp_eval_train
        model_eval_test = temp_eval_test
    return model_eval_train, model_eval_test, cf_matrix_train, cf_matrix_test

In [3]:
path=r'data'

In [4]:
X_train = np.load(path + "/X_train.npy")
y_train = np.load(path + "/y_train.npy")
X_test = np.load(path + "/X_test.npy")
y_test = np.load(path + "/y_test.npy")

In [5]:
model_eval_train = pd.DataFrame({},[])
model_eval_test = pd.DataFrame({},[])

In [7]:
knn_dtw = KNeighborsTimeSeriesClassifier(distance="dtw",n_neighbors=20,weights='uniform',leaf_size=30,n_jobs=-1)
tree = ShapeletTreeClassifier(random_state=10,n_shapelets=50,min_shapelet_size=0,max_shapelet_size=1,metric="euclidean")
muse= MUSE(window_inc=10, use_first_order_differences=False,support_probabilities=True)
rocket = RocketClassifier(num_kernels=2000,n_jobs=-1)
hc2=HIVECOTEV2(time_limit_in_minutes=0.25)

In [8]:
models={
    'knn_dtw':knn_dtw,
    'tree':tree,
    'muse':muse,
    'rocket':rocket,
    'hc2':hc2}

In [9]:
for i in range(5):
    print(i)
    for item in models.items():
        name=item[0]
        model=item[1]
        h = model.fit(X_train,y_train)
        model_eval_train, model_eval_test, cf_matrix_train, cf_matrix_test = model_eval_data(model, X_train, y_train, 
                                                            X_test, y_test, 
                                                            model_eval_train, 
                                                            model_eval_test,
                                                            Name= str(i)+'-'+name)

0


c:\Users\xyang\miniconda3\envs\tt\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 20.
  warnings.warn(


1


c:\Users\xyang\miniconda3\envs\tt\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 20.
  warnings.warn(


2


c:\Users\xyang\miniconda3\envs\tt\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 20.
  warnings.warn(


3


c:\Users\xyang\miniconda3\envs\tt\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 20.
  warnings.warn(


4


c:\Users\xyang\miniconda3\envs\tt\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 20.
  warnings.warn(


In [11]:
model_eval_test

,precision,recall,f1-score,accuracy,ROC,PR,MCC
0-knn_dtw,0.947368,0.156522,0.268657,0.735135,0.767570,0.656137,0.319995
0-tree,0.434783,0.434783,0.434783,0.648649,0.589940,0.364712,0.179881
0-muse,0.831169,0.556522,0.666667,0.827027,0.850230,0.785623,0.576364
0-rocket,0.805825,0.721739,0.761468,0.859459,0.821654,0.668082,0.664299
0-hc2,0.930233,0.695652,0.796020,0.889189,0.935090,0.898755,0.736477
1-knn_dtw,0.947368,0.156522,0.268657,0.735135,0.767570,0.656137,0.319995
1-tree,0.434783,0.434783,0.434783,0.648649,0.589940,0.364712,0.179881
1-muse,0.765432,0.539130,0.632653,0.805405,0.851117,0.770460,0.520027
1-rocket,0.793814,0.669565,0.726415,0.843243,0.795567,0.634213,0.622068
1-hc2,0.744681,0.608696,0.669856,0.813514,0.850810,0.765796,0.547081


In [ ]:
model_eval_test.to_csv('c2.csv')

In [10]:
model_eval_test

,precision,recall,f1-score,accuracy,ROC,PR,MCC
0-knn_dtw,0.947368,0.156522,0.268657,0.735135,0.767570,0.656137,0.319995
0-tree,0.434783,0.434783,0.434783,0.648649,0.589940,0.364712,0.179881
0-muse,0.831169,0.556522,0.666667,0.827027,0.850230,0.785623,0.576364
0-rocket,0.805825,0.721739,0.761468,0.859459,0.821654,0.668082,0.664299
0-hc2,0.930233,0.695652,0.796020,0.889189,0.935090,0.898755,0.736477
1-knn_dtw,0.947368,0.156522,0.268657,0.735135,0.767570,0.656137,0.319995
1-tree,0.434783,0.434783,0.434783,0.648649,0.589940,0.364712,0.179881
1-muse,0.765432,0.539130,0.632653,0.805405,0.851117,0.770460,0.520027
1-rocket,0.793814,0.669565,0.726415,0.843243,0.795567,0.634213,0.622068
1-hc2,0.744681,0.608696,0.669856,0.813514,0.850810,0.765796,0.547081
